In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [30]:
from collections import defaultdict
N, M = map(int, input().split())

G = [list() for _ in range(N)]
GT = [defaultdict(int) for _ in range(N)]
GB = [defaultdict(int) for _ in range(N)]

for _ in range(M):
    a, b, c = map(int, input().split())
    a -= 1
    b -= 1
    G[a].append(b)
    G[b].append(a)
    GT[a][b] = c
    GB[b][a] = 0

ans = 0
while 1:
    _min = 10 ** 20
    v = [False] * N
    flag = False
    cost = []
    
    def dfs(x):
        global flag
        global _min
        global ans
        
        v[x] = True
        if x == N-1:
            flag = True
            _min = min(cost)
            ans += _min
            return 
        
        for nex in G[x]:
            if v[nex]:
                continue
                
            if GT[x][nex]:
                cost.append(GT[x][nex])
                dfs(nex)
                if flag:
                    GT[x][nex] -= _min
                    GB[nex][x] += _min
                    return
                cost.pop()
                    
            if GB[x][nex]:
                cost.append(GB[x][nex])
                dfs(nex)
                if flag:
                    GB[x][nex] -= _min
                    GT[nex][x] += _min
                    return
                cost.pop()
        return 
    
    dfs(0)
    if not flag:
        break
    
print(ans)

8


In [27]:
GT

[defaultdict(int, {1: 0, 3: 1}),
 defaultdict(int, {2: 1, 4: 5}),
 defaultdict(int, {5: 0}),
 defaultdict(int, {4: 0}),
 defaultdict(int, {5: 0, 3: 0, 1: 0}),
 defaultdict(int, {})]

In [28]:
GB

[defaultdict(int, {1: 0, 3: 0}),
 defaultdict(int, {0: 5, 2: 0}),
 defaultdict(int, {1: 3, 5: 0}),
 defaultdict(int, {0: 3, 4: 0}),
 defaultdict(int, {1: 2, 3: 3}),
 defaultdict(int, {2: 3, 4: 5})]